### importing dependaices

In [1]:
import pandas as pd
import requests
import json
from pprint import pprint

### importing api key

In [2]:
from config import api_key

### reading in data from csv
#### filtered_fires.csv was filtered from origonal data in SQLiteStudio removing low/nominal confidence and non-vegetation fires

In [3]:
data = pd.read_csv (r'C:\Users\Ryan\Desktop\forest_fires\filtered_fires.csv')

### Snippet from readme file explaining the column values

https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/v1-vnp14imgt#ed-viirs-375m-attributes

Latitude: Center of nominal 375 m fire pixel

Longitude: Longitude	Center of nominal 375 m fire pixel

Bright_ti4: Brightness temperature I-4	VIIRS I-4 channel brightness temperature of the fire pixel measured in Kelvin.

Scan: Along Scan pixel size	The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.

Track: Along Track pixel size	The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.

Acq_Date: Acquisition Date	Date of VIIRS acquisition.
Acq_Time: Acquisition Time	Time of acquisition/overpass of the satellite (in UTC).
Satellite: Satellite	N= Suomi National Polar-orbiting Partnership (Suomi NPP)
Confidence: Confidence	
This value is based on a collection of intermediate algorithm quantities used in the detection process. It is intended to help users gauge the quality of individual hotspot/fire pixels. Confidence values are set to low, nominal and high. Low confidence daytime fire pixels are typically associated with areas of sun glint and lower relative temperature anomaly (<15K) in the mid-infrared channel I4. Nominal confidence pixels are those free of potential sun glint contamination during the day and marked by strong (>15K) temperature anomaly in either day or nighttime data. High confidence fire pixels are associated with day or nighttime saturated pixels.

Version: (Collection and source) Version identifies the collection (e.g. VIIRS Collection 1) and source of data processing: Near Real-Time (NRT suffix added to collection) or Standard Processing (collection only). "1.0NRT" - Collection 1 NRT processing. "1.0" - Collection 1 Standard processing.

Bright_ti5:	Brightness temperature I-5	I-5 Channel brightness temperature of the fire pixel measured in Kelvin.

FRP	(Fire Radiative Power): FRP depicts the pixel-integrated fire radiative power in MW (megawatts). Given the unique spatial and spectral resolution of the data, the VIIRS 375 m fire detection algorithm was customized and tuned in order to optimize its response over small fires while balancing the occurrence of false alarms. Frequent saturation of the mid-infrared I4 channel (3.55-3.93 µm) driving the detection of active fires requires additional tests and procedures to avoid pixel classification errors. As a result, sub-pixel fire characterization (e.g., fire radiative power [FRP] retrieval) is only viable across small and/or low-intensity fires. Systematic FRP retrievals are based on a hybrid approach combining 375 and 750 m data. In fact, starting in 2015 the algorithm incorporated additional VIIRS channel M13 (3.973-4.128 µm) 750 m data in both aggregated and unaggregated format.

Type:Inferred hot spot type	
    0 = presumed vegetation fire
    1 = active volcano
    2 = other static land source
    3 = offshore detection (includes all detections over water)
DayNight: Day or Night; D= Daytime fire, N= Nighttime fire

### creating the dataframe from csv file with header names

In [4]:
df = pd.DataFrame(data, columns = ['latitude',
                                   'longitude',
                                   'bright_ti4',
                                   'scan',
                                   'track',
                                   'acq_date',
                                   'acq_time',
                                   'satellite',
                                   'instrument',
                                   'confidence',
                                   'version',
                                   'bright_ti5',
                                   'frp',
                                   'daynight',
                                   'type'])
filtered_df = df.drop(['bright_ti4',
                       'scan',
                       'track',
                       'satellite',
                       'instrument',
                       'confidence',
                       'version',
                       'bright_ti5',
                       'daynight',
                       'type'], axis = 1)

In [5]:
filtered_df.head

<bound method NDFrame.head of         latitude   longitude    acq_date  acq_time    frp
0      19.418756 -155.083542  01/10/2015      1120   7.14
1      19.408396 -155.091934  01/10/2015      1120   8.82
2      19.413961 -155.091431  01/10/2015      1120  24.02
3      19.412861 -155.093277  01/10/2015      1120  22.31
4      19.421993 -155.060715  01/10/2015      1120  14.90
...          ...         ...         ...       ...    ...
59859  27.412804  -81.409821  11/11/2017      1752   9.22
59860  27.412285  -81.410515  11/11/2017      1752  10.40
59861  30.525211  -86.703514  11/11/2017      1753  12.33
59862  30.526947  -86.697243  11/11/2017      1753  12.33
59863  26.298752  -97.524200  11/11/2017      1933  11.39

[59864 rows x 5 columns]>

### rounding the lat and lon to simplify and group dataframe

In [6]:
rounded_df = filtered_df.round({'latitude':1, 'longitude':1})

In [7]:
rounded_df.head

<bound method NDFrame.head of        latitude  longitude    acq_date  acq_time    frp
0          19.4     -155.1  01/10/2015      1120   7.14
1          19.4     -155.1  01/10/2015      1120   8.82
2          19.4     -155.1  01/10/2015      1120  24.02
3          19.4     -155.1  01/10/2015      1120  22.31
4          19.4     -155.1  01/10/2015      1120  14.90
...         ...        ...         ...       ...    ...
59859      27.4      -81.4  11/11/2017      1752   9.22
59860      27.4      -81.4  11/11/2017      1752  10.40
59861      30.5      -86.7  11/11/2017      1753  12.33
59862      30.5      -86.7  11/11/2017      1753  12.33
59863      26.3      -97.5  11/11/2017      1933  11.39

[59864 rows x 5 columns]>

### grouping by date/lat/lon and returning max FRP to get the highest FRP for unique day/location

In [8]:
df2 = rounded_df.groupby(['latitude', 'longitude', 'acq_date'])['frp'].transform(max) == df['frp']

In [9]:
df3 = rounded_df[df2]

In [10]:
print(df3)

       latitude  longitude    acq_date  acq_time     frp
2          19.4     -155.1  01/10/2015      1120   24.02
11         32.0      -91.7  01/10/2015      1911   81.70
12         30.8      -95.0  01/10/2015      1911   69.37
13         31.1      -90.8  01/10/2015      1911  128.65
14         31.1      -90.8  01/10/2015      1911  128.65
...         ...        ...         ...       ...     ...
59858      26.5      -80.5  11/11/2017      1752   14.68
59860      27.4      -81.4  11/11/2017      1752   10.40
59861      30.5      -86.7  11/11/2017      1753   12.33
59862      30.5      -86.7  11/11/2017      1753   12.33
59863      26.3      -97.5  11/11/2017      1933   11.39

[23637 rows x 5 columns]


### dropping duplicate data

In [11]:
df4 = df3.drop_duplicates()

### condensed forest fire dataframe

#### consists of single FRP value for each simplified location
#### need to left-join on index long-form lat/long from 'filtered_df' to do marker placement
#### final DF should also have formatted location data (reverse google-API call) for marker labels

In [12]:
print(df4)

       latitude  longitude    acq_date  acq_time     frp
2          19.4     -155.1  01/10/2015      1120   24.02
11         32.0      -91.7  01/10/2015      1911   81.70
12         30.8      -95.0  01/10/2015      1911   69.37
13         31.1      -90.8  01/10/2015      1911  128.65
15         35.8      -92.0  01/10/2015      1912   22.05
...         ...        ...         ...       ...     ...
59857      19.3     -155.0  11/11/2017      1143    6.84
59858      26.5      -80.5  11/11/2017      1752   14.68
59860      27.4      -81.4  11/11/2017      1752   10.40
59861      30.5      -86.7  11/11/2017      1753   12.33
59863      26.3      -97.5  11/11/2017      1933   11.39

[20992 rows x 5 columns]


### transforming dataframe to dictionary to execute for-loops
#### perform API calls - formatted address, marker placement

In [13]:
fire_dict = df4.to_dict()
print(fire_dict)

{'latitude': {2: 19.4, 11: 32.0, 12: 30.8, 13: 31.1, 15: 35.8, 16: 35.7, 17: 34.9, 18: 36.0, 19: 34.1, 20: 33.1, 22: 34.2, 23: 35.5, 24: 35.3, 25: 34.3, 26: 34.8, 28: 38.1, 30: 36.1, 31: 36.7, 48: 19.4, 51: 28.8, 54: 31.1, 55: 27.2, 56: 33.9, 57: 31.2, 67: 34.9, 69: 28.8, 70: 28.3, 71: 32.8, 72: 39.2, 74: 39.2, 77: 35.7, 78: 36.6, 79: 46.1, 82: 46.4, 84: 46.2, 86: 48.3, 87: 35.8, 89: 19.4, 91: 30.0, 92: 29.9, 93: 31.0, 95: 31.0, 97: 30.9, 102: 31.1, 103: 38.8, 107: 35.6, 110: 32.8, 111: 34.8, 114: 37.9, 116: 35.1, 117: 35.1, 118: 35.7, 120: 40.7, 121: 36.6, 122: 36.6, 125: 36.3, 126: 38.1, 127: 30.9, 128: 35.6, 131: 37.9, 132: 39.2, 135: 36.2, 136: 39.3, 137: 46.1, 139: 46.1, 141: 47.7, 142: 46.8, 143: 19.4, 146: 34.3, 147: 34.2, 149: 34.1, 150: 27.3, 151: 30.8, 152: 30.5, 153: 29.7, 154: 30.2, 155: 33.4, 157: 34.3, 158: 34.5, 159: 35.7, 160: 35.5, 161: 34.5, 162: 35.1, 165: 34.4, 166: 33.7, 167: 34.1, 169: 35.8, 170: 36.1, 171: 36.5, 172: 44.2, 173: 47.7, 174: 46.0, 176: 46.8, 177: 48

### reverse geodata api call to get location based on lat/lon

#### need to figure out for loop to get formatted address for markers
#### use long-form (4 sig-dig's) for lat/lon

In [14]:
test_lat = 41.563
test_long = -84.048

In [15]:
target_url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={test_lat},{test_long}&key={api_key}'

In [16]:
geo_data = requests.get(target_url).json()
pprint(geo_data)

{'plus_code': {'compound_code': 'HX72+5Q Delta, OH, USA',
               'global_code': '86HQHX72+5Q'},
 'results': [{'address_components': [{'long_name': 'Unnamed Road',
                                      'short_name': 'Unnamed Road',
                                      'types': ['route']},
                                     {'long_name': 'Delta',
                                      'short_name': 'Delta',
                                      'types': ['locality', 'political']},
                                     {'long_name': 'York Township',
                                      'short_name': 'York Township',
                                      'types': ['administrative_area_level_3',
                                                'political']},
                                     {'long_name': 'Fulton County',
                                      'short_name': 'Fulton County',
                                      'types': ['administrative_area_level_2',
           

In [17]:
formatted_address = (geo_data["results"][1]["formatted_address"])

In [18]:
print(formatted_address)

York Township, OH, USA
